In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

# Load your preprocessed dataset
file_path = 'preprocessed_tweets_dataset.csv'
tweets_df = pd.read_csv(file_path)

# Handle missing values if any
tweets_df['preprocessed_text'] = tweets_df['preprocessed_text'].fillna('')

# Split dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(tweets_df['preprocessed_text'],
                                                                    tweets_df['Sentiment'],
                                                                    test_size=0.2,
                                                                    random_state=42)

# Example of cleaning labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
train_labels = train_labels.map(sentiment_mapping)
val_labels = val_labels.map(sentiment_mapping)

# Convert to list of strings
train_texts_list = train_texts.astype(str).tolist()
val_texts_list = val_texts.astype(str).tolist()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Encode datasets using batch_encode_plus
train_encodings = tokenizer.batch_encode_plus(train_texts_list, truncation=True, padding=True)
val_encodings = tokenizer.batch_encode_plus(val_texts_list, truncation=True, padding=True)

# Convert encodings to TensorFlow Dataset format
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: np.array(value) for key, value in train_encodings.items()},
    np.array(train_labels)
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: np.array(value) for key, value in val_encodings.items()},
    np.array(val_labels)
))

# Define model
model_name = 'bert-base-uncased'
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',                # output directory
    evaluation_strategy='epoch',           # evaluation strategy to adopt
    per_device_train_batch_size=8,         # batch size for training
    per_device_eval_batch_size=16,         # batch size for evaluation
    logging_dir='./logs',                  # directory for storing logs
    save_total_limit=3,                    # limit the total amount of checkpoints
    learning_rate=2e-5,                    # learning rate
    num_train_epochs=3,                    # number of training epochs
    weight_decay=0.01,                     # strength of weight decay
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {'accuracy': np.argmax(p.predictions, axis=1),
                               'class_report': classification_report(val_labels, np.argmax(p.predictions, axis=1))}
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


AttributeError: 'TFBertForSequenceClassification' object has no attribute 'to'